https://python.langchain.com/v0.2/docs/integrations/tools/sql_database/
https://github.com/langchain-ai/langchain/blob/master/cookbook/sql_db_qa.mdx

In [1]:
import sqlite3
import os

if not os.path.exists("Chinook.db"):
    # 데이터베이스 연결 생성
    conn = sqlite3.connect('Chinook.db')

    # SQL 파일 읽기
    with open('Chinook_Sqlite.sql', 'r', encoding='utf-8') as sql_file:
        sql_script = sql_file.read()

    # SQL 명령 실행
    cursor = conn.cursor()
    cursor.executescript(sql_script)
    
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    print("생성된 테이블 목록:")
    for table in tables:
        print(table[0])

    # 변경사항 저장 및 연결 종료
    conn.commit()
    conn.close()

    print("Chinook.db 생성이 완료되었습니다.")

In [3]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4", temperature=0)

In [4]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [5]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=model)

In [6]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

E:\git\langchain-test\.venv\lib\site-packages\langsmith\client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


['dialect', 'top_k']


E:\git\langchain-test\.venv\lib\site-packages\langsmith\client.py:5434: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [7]:
system_message = prompt_template.format(dialect="PostgreSQL", top_k=5)

In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(
    model, toolkit.get_tools(), state_modifier=system_message
)

In [10]:
example_query = ("국가, 도시별 인보이스 총 금액의 합계를 알려줘")

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

국가, 도시별 인보이스 총 금액의 합계를 알려줘
================================== Ai Message ==================================

먼저, 데이터베이스에 어떤 테이블들이 있는지 확인해야 합니다. 이를 위해 sql_db_list_tables 도구를 사용하겠습니다.
Tool Calls:
  sql_db_list_tables (call_6TFj91Z4PYjZWjhGqnSjmQnS)
 Call ID: call_6TFj91Z4PYjZWjhGqnSjmQnS
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================

데이터베이스에는 다음과 같은 테이블들이 있습니다: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track.

국가와 도시별 인보이스 총 금액의 합계를 구하기 위해서는 'Customer' 테이블에서 국가와 도시 정보를 가져오고, 'Invoice' 테이블에서 인보이스 금액을 가져와야 할 것 같습니다. 이 두 테이블의 스키마를 확인해 보겠습니다.
Tool Calls:
  sql_db_schema (call_3uUnNT8zVYO7NokeiH7pcsKh)
 